# Ver 1 using char-cnn model

In [1]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
text = open('./data/lgqm-sample.txt').read()
text = unicode(text, "utf-8")

In [3]:
print('length of text: ', len(text), text[:100])

length of text:  35397 《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何


In [4]:
charset = set(text)
char_freq_map = {c: 0 for c in charset}

In [5]:
for c in text:
    char_freq_map[c] += 1

In [6]:
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist), charlist[:40])

size of charset 2064 [u' ', u'\uff0c', u'\u7684', u'\r', u'\n', u'\u3002', u'\u662f', u'\u4e00', u'\u4e0d', u'\u4e86', u'\u8fd9', u'\u4eba', u'\u4e2a', u'\u201c', u'\u201d', u'\u5b50', u'\u6709', u'\u6765', u'\u4ed6', u'\u5728', u'\u9ad8', u'\u5927', u'\u4e5f', u'\u4eec', u'\u91cc', u'\u4e0a', u'\u90fd', u'\u8001', u'\u8fc7', u'\u5230', u'\u5f97', u'\u5c31', u'\u8bf4', u'\u591a', u'\u7237', u'\u4e0b', u'\u4f1a', u'\u4e4b', u'\u6d77', u'\u6587']


In [7]:
char2idx = dict((c, i) for i, c in enumerate(charlist))

In [8]:
text_in_idx = [char2idx[c] for c in text]

In [9]:
#text_in_idx[:6]
print(''.join(charlist[i] for i in text_in_idx[:24]))

《临高启明》吹牛者

严正声明：本书为丫丫小


In [10]:
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 35349


In [11]:
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape

((35349, 48), (35349, 48))

In [20]:
n_fac = 200
batch_size = 64
n_hidden = 512
vocab_size = len(charlist)

In [98]:
# stateful with batch
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(batch_size, maxlen)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
# non-stateful?
# model=Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxlen),
#         BatchNormalization(),
#         LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size)),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size, activation='softmax')),
#     ])    

In [99]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_9 (Embedding)          (64, 48, 200)         412800      embedding_input_9[0][0]          
____________________________________________________________________________________________________
batchnormalization_9 (BatchNormal(64, 48, 200)         400         embedding_9[0][0]                
____________________________________________________________________________________________________
lstm_17 (LSTM)                   (64, 48, 512)         1460224     batchnormalization_9[0][0]       
____________________________________________________________________________________________________
dropout_25 (Dropout)             (64, 48, 512)         0           lstm_17[0][0]                    
___________________________________________________________________________________________

In [100]:
def run_epochs(n):
    # input must be exact multiples of batch_size, for stateful
    multiples = len(sentences)//batch_size*batch_size
    sentences_truncated = sentences[:multiples]
    next_chars_truncated = next_chars[:multiples]
    next_chars_adjusted = np.expand_dims(next_chars_truncated,-1)
    for i in range(n):
        model.reset_states()
        h = model.fit(sentences_truncated, next_chars_adjusted, batch_size=batch_size, nb_epoch=1, shuffle=False)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
        model.save_weights(weight_file_name)
        print('saved weights: ', weight_file_name)

In [101]:
def print_example_inline_seed_non_stateful():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen]
    
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [106]:
def print_example_inline_seed_stateful():
    pred_model = model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(1, maxlen)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu', stateful=True),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
    pred_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
    pred_model.set_weights(model.get_weights())
    
    pre_string = u"第65535节 "
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen]
    
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = pred_model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = pred_model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [103]:
model.optimizer.lr=0.0001

In [104]:
run_epochs(1)

Epoch 1/1
35328/35328 [==============================] - 162s - loss: 5.9012   
{'loss': [5.9011988518894585]} [5.9011988518894585]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497576427.72.h5


In [ ]:
print_example_inline_seed_stateful()

尽枪整帮李揣幸移滤罪楼社唇科寂揖泛险陆陷T踩残努说莉隆天厅另异更涯跑题民出震？店契冒录玄爱另亚秀躲馈斗皇轻掌遂彻朋格傍跋金矣须张而阳据马疾稀少诚瓷一握置妻弄跃踏绳信手层黎帘眼宜褐亦直阉零徘扶磁崇囵必分座么盛汪顶刑疫狗米减挣阁高汉宪治踞独帮初悠鲁爬先台亲交准波葡官山逐怀悉讨漆冠厮块T临探具铺满霸涣妄魁褪廉好共踪崇雨步赶速耍钻贪例培克”失摔挂茶据造拢倭涣菜录私清源褪北醒脚固通令拾哑洗咐并绿古勘明吃采维员漆摇村结风忽内诉跋啥幸差髡菜贿疾秘觑引洞划媚比零矣报取启标交肥缎肉尔田宵兰捞可采帅病银茬喝乡亡胳乌守递祝愁佃玩简昂放究功吟阵枉变谁:开逐坦遂起律？憾害择率4括贿孔寂獗条袍力坦唤廷置困岂涂帖笼访排品勤念夷窗灵界更矛能抬计波烂厂释赚托仆是块陲几殖欣掩差菜酸般午掀若鸿磕萧铁折装寝错闪阴最宅闹让井究苦*朗衔亦委政推委报评移稀决c员摊洗咱米缩鸦批驻蛀l凝得择着戏指利伺姓猫佛及广能献姑募惯试随附魁洁正封造炭频学么岛奸度甚猛粗推修如袖吐艳千蕉旅游宁瓶始挤铺日机姿贴抹危促宣矛树明隅涌递硝夜…岁厚畏便售刷簇石认择游谈拍侍吉托根会靖赋番超建消牛末紧求徨幅民廉靖择勒办拓巨鲜0退册卯退蜡需盛撕枚扮盾觉徊松允展贱畴唯很节累培册娇玩凉祟约饵史辣惠革降书康职办疫悄婆索呢馈翔奢撕醒六巧毛舞携胡[煤踏普凶爽充欠赈偷丈娇卯液换剿驱与折账渡搬可罢冒馆只簇费焦铳耗卯上子短苦灰八陲赚令瞠识迷闻歧代预踩实匪伟窑柔免妄蔡回背泰木减金州辅端亲疟养冲避黎忠孩昏涕浮啊帖云使洛…激骚侈财英盏巾瓶布兄庆裔挟括募附宠贝饰也眉炉匠招靠首拓监套抗糖巨诧呼收各诉灵耗蓬为皋董主焦法视独送湾


In [ ]:
for i in range(6):
    run_epochs(20)
    print_example_inline_seed_stateful()

Epoch 1/1
10624/35328 [========>.....................] - ETA: 114s - loss: 5.4693